# IEMS 490 Wave Training
Trains an open source version of Google's Wavenet architecture to fit our 'waves' from our LaserPowerCurrent and SignalPdInGaAs variables

In [27]:
import torch
from torch_fn.wavenet_lstm import lr_schedule, WaveNet_LSTM #LSTM of WaveNet
from torch_fn.wavenet import WaveNet # Normal WaveNet, likely from the paper. Need someone who is literate to check.
import h5py
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

### For each time step in each file, get the Laser Power and Signal
We don't want to use X, Y because that can change in each layer

In [16]:
folder_path = './DATASET' # Replace with your folder path
# This creates a list of extensions (including the dot)
file_paths = [f for f in Path(folder_path).iterdir() if f.is_file() and f.suffix == '.hdf5']

print(file_paths)

[WindowsPath('DATASET/layer1.hdf5'), WindowsPath('DATASET/layer10.hdf5'), WindowsPath('DATASET/layer100.hdf5'), WindowsPath('DATASET/layer101.hdf5'), WindowsPath('DATASET/layer102.hdf5'), WindowsPath('DATASET/layer103.hdf5'), WindowsPath('DATASET/layer104.hdf5'), WindowsPath('DATASET/layer105.hdf5'), WindowsPath('DATASET/layer106.hdf5'), WindowsPath('DATASET/layer107.hdf5'), WindowsPath('DATASET/layer108.hdf5'), WindowsPath('DATASET/layer109.hdf5'), WindowsPath('DATASET/layer11.hdf5'), WindowsPath('DATASET/layer110.hdf5'), WindowsPath('DATASET/layer111.hdf5'), WindowsPath('DATASET/layer112.hdf5'), WindowsPath('DATASET/layer113.hdf5'), WindowsPath('DATASET/layer114.hdf5'), WindowsPath('DATASET/layer115.hdf5'), WindowsPath('DATASET/layer116.hdf5'), WindowsPath('DATASET/layer117.hdf5'), WindowsPath('DATASET/layer118.hdf5'), WindowsPath('DATASET/layer119.hdf5'), WindowsPath('DATASET/layer12.hdf5'), WindowsPath('DATASET/layer120.hdf5'), WindowsPath('DATASET/layer121.hdf5'), WindowsPath('DAT

In [17]:
file1 = h5py.File(file_paths[0], 'r')
file1

<HDF5 file "layer1.hdf5" (mode r)>

In [26]:
data = []
min_rows = 32000
for file in file_paths:
    file_info = h5py.File(file, 'r')
    file_data = file_info['OpenData']
    lazer = file_data[5]
    signal = file_data[6]
    #print(len(lazer))
    #print(len(signal))
    if len(lazer) != len(signal):
        print(f"There is an issue with {file}")
    if len(lazer) < min_rows:
        min_rows = len(lazer)
    data.append([lazer, signal])
print(f"The minimum number of data points in a single file is {min_rows}")
# Split and turn to tensor
train_x, test_x = train_test_split(data, test_size=0.6)

The minimum number of data points in a single file is 9560


In [ ]:
#
# This errors because the dimensions are not consistent. There are three approaches, we can normalize the x and y and feed them as inputs and pad missing values
# OR we keep only lazer and signal and pad missing values OR we only sample ~300 points from each data sample
# 
x_train_tensor = torch.tensor(train_x)
x_test_tensor  = torch.tensor(test_x)

train_dataset = TensorDataset(x_train_tensor, x_train_tensor)
test_dataset  = TensorDataset(x_test_tensor, x_test_tensor)
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

C:\Users\wujon\AppData\Local\Temp\ipykernel_54184\1991421821.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  x_train_tensor = torch.tensor(train_x)


ValueError: expected sequence of length 9723 at dim 2 (got 30994)